In [1]:
import pandas as pd
import numpy as np

#Reading the 'plays' dataset, turning all items into integers
users = pd.DataFrame(np.array([i.strip().split("\t") for i in open('lastfm_data/user_artists.dat', 'r', encoding='latin-1').readlines()]),
                       columns = ['userID', 'artistID', 'plays'])
users = users.drop(users.index[0])
users = users.apply(pd.to_numeric)
users.head()

#Reading 'artists' dataset, splitting the first column into three columns on the newline, turning artistid into int
artists = pd.DataFrame(open('lastfm_data/artists.dat', 'r',  encoding='latin-1').readlines())
artists = pd.DataFrame(artists[0].str.split("\t").values.tolist(),columns = ['artistID', 'name', 'url', 'pictureURL'])
artists = artists.drop(artists.index[0])
artists['artistID'] = artists['artistID'].apply(pd.to_numeric)
artists = artists.drop(columns = ['url', 'pictureURL'])

In [2]:
artists.head()

,artistID,name
1,1,MALICE MIZER
2,2,Diary of Dreams
3,3,Carpathian Forest
4,4,Moi dix Mois
5,5,Bella Morte


In [3]:
A = users.pivot(index = 'userID', columns ='artistID', values = 'plays').fillna(0)

In [4]:
A_vals = A.values
ratings_mean = np.mean(A_vals, axis = 1)
A_norm = A_vals - ratings_mean.reshape(-1, 1)
A_norm[:10]

array([[-9.56992967, -9.56992967, -9.56992967, ..., -9.56992967,
        -9.56992967, -9.56992967],
       [-1.16271552, -1.16271552, -1.16271552, ..., -1.16271552,
        -1.16271552, -1.16271552],
       [-1.52733666, -1.52733666, -1.52733666, ..., -1.52733666,
        -1.52733666, -1.52733666],
       ...,
       [-1.45173548, -1.45173548, -1.45173548, ..., -1.45173548,
        -1.45173548, -1.45173548],
       [-1.61768376, -1.61768376, -1.61768376, ..., -1.61768376,
        -1.61768376, -1.61768376],
       [-2.43421053, -2.43421053, -2.43421053, ..., -2.43421053,
        -2.43421053, -2.43421053]])

In [5]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(A_norm, k = 50)

In [6]:
print(sigma, sigma.shape)
sigma = np.diag(sigma)
sigma, sigma.shape

[ 53152.325196    54617.21501538  55053.55601408  55674.11736015
  56428.86911665  57149.17061034  57251.10442122  58263.90660131
  58808.97338726  59930.78126213  61956.5194954   64181.01701669
  67665.18963723  68714.5581343   69637.2346044   70194.84155807
  72979.04715761  77421.04229318  79502.62218135  81993.17103901
  84487.95159216  86937.37316232  96108.78635272  96609.97240506
  99884.94765218 101301.09459364 103000.98267071 103502.1770617
 109469.49320176 114535.95034243 115498.3513451  121365.37000331
 124723.73606083 132824.51859405 134758.71833846 140273.58505092
 146172.98121334 151172.54630454 171161.84123895 179649.27672706
 183282.2335299  232682.90111541 247464.67255507 255387.53438738
 258227.2051966  269388.22289065 317571.78846536 323526.36832677
 324895.90434521 405527.41904943] (50,)


(array([[ 53152.325196  ,      0.        ,      0.        , ...,
              0.        ,      0.        ,      0.        ],
        [     0.        ,  54617.21501538,      0.        , ...,
              0.        ,      0.        ,      0.        ],
        [     0.        ,      0.        ,  55053.55601408, ...,
              0.        ,      0.        ,      0.        ],
        ...,
        [     0.        ,      0.        ,      0.        , ...,
         323526.36832677,      0.        ,      0.        ],
        [     0.        ,      0.        ,      0.        , ...,
              0.        , 324895.90434521,      0.        ],
        [     0.        ,      0.        ,      0.        , ...,
              0.        ,      0.        , 405527.41904943]]), (50, 50))

In [7]:
predictions = np.dot(np.dot(U, sigma), Vt) + ratings_mean.reshape(-1, 1)

In [8]:
predictions_df = pd.DataFrame(predictions, columns = A.columns)

In [9]:
predictions_df.iloc[99]

artistID
1         7.986171
2         9.453314
3         7.910007
4         7.927932
5         8.130232
6        11.741285
7        99.885753
8        17.094859
9         7.349655
10        7.990935
11        8.276860
12       18.164593
13        7.274780
15       86.286881
16        7.979341
17        7.531298
18        8.290999
19       10.517993
20        7.903877
21        7.902586
22        7.903243
23        9.544148
24        7.903584
25       34.242653
26        7.260900
27       10.220790
28        7.946332
30       -3.584157
31        7.755359
32       10.735594
           ...    
18713     7.903136
18714     7.903143
18715     7.903164
18716     7.903169
18717     7.903170
18718     7.903170
18719     7.903182
18720     7.903187
18721     7.903191
18722     7.903192
18723     7.903209
18724     7.903212
18725     7.907417
18726     7.905122
18727     7.904904
18728     7.904817
18729     7.904811
18730     7.904719
18734     7.856964
18735     7.863113
18736     7.863831
187

In [ ]:
def recommender(predictions_df, UID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row = UID - 1 # UID starts at 1, not 0
    sorted_predictions = predictions_df.iloc[user_row].sort_values(ascending=False) 
    
    # Get the original user data and merge in the movie information 
    user_data = original_ratings_df[original_ratings_df.userID == (UID)]
    user_full = user_data.merge(movies_df, how = 'left', left_on = 'artistID', right_on = 'artistID').sort_values(['plays'], ascending=False)

    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = movies_df[~movies_df['artistID'].isin(user_full['artistID'])] \
                      .merge(pd.DataFrame(sorted_predictions).reset_index(), 
                             how = 'left', left_on = 'artistID', right_on = 'artistID') \
                      .rename(columns = {user_row: 'Predictions'}) \
                      .sort_values('Predictions', ascending = False) \
                      .iloc[:num_recommendations, :-1]
                    
    print ('User {0} has already rated {1} movies.'.format(UID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    return user_full, recommendations

In [ ]:
recommender(predictions_df, 500, artists, users, num_recommendations=5)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.neighbors import NearestNeighbors

In [11]:
users_artists = pd.merge(artists, users, on='artistID', how='left')

In [12]:
users_artists.head()

,artistID,name,userID,plays
0,1,MALICE MIZER,34,212
1,1,MALICE MIZER,274,483
2,1,MALICE MIZER,785,76
3,2,Diary of Dreams,135,1021
4,2,Diary of Dreams,257,152


In [13]:
pivot_data = users_artists.pivot(index = 'userID', columns = 'name', values = 'plays').fillna(0)

In [14]:
pivot_data.head()

name,!!!,!DISTAIN,!deladap,"""Weird Al"" Yankovic",#####,#2 Orchestra,$lick,$lick feat. Six-Fo,(1962-1966),(Pre) Kingdom,...,í°ìë¼,íì,ï¼¬ï½ï¼¬ï½,ï¼­ï¼¡ï¼®ï¼®ï¼¥ï¼±ï¼µï¼©ï¼®,ï¼°ï¼¥ï¼²ï¼¥ï¼³ï¼´ï¼²ï¼¯ï¼©ï¼«ï¼¡,ï¼²ï¼¥ï¼¤,ï¼¶ï½ï½ï½ ï½ï½ï¼¡,ï½ï½ï½ï½ï½ï½ ï½ï½ï½,ï½ï½ï½ï½,ï½ï½ï½ï½ï½ï½ï½
userID,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
pivot_data.loc[pivot_data['Lady Gaga'] > 1]

name,!!!,!DISTAIN,!deladap,"""Weird Al"" Yankovic",#####,#2 Orchestra,$lick,$lick feat. Six-Fo,(1962-1966),(Pre) Kingdom,...,í°ìë¼,íì,ï¼¬ï½ï¼¬ï½,ï¼­ï¼¡ï¼®ï¼®ï¼¥ï¼±ï¼µï¼©ï¼®,ï¼°ï¼¥ï¼²ï¼¥ï¼³ï¼´ï¼²ï¼¯ï¼©ï¼«ï¼¡,ï¼²ï¼¥ï¼¤,ï¼¶ï½ï½ï½ ï½ï½ï¼¡,ï½ï½ï½ï½ï½ï½ ï½ï½ï½,ï½ï½ï½ï½,ï½ï½ï½ï½ï½ï½ï½
userID,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
player_counts = pivot_data.sum(axis =0, skipna = True).sort_values(ascending=False)

In [ ]:
player_counts

In [18]:
pivot_data[pivot_data >= 1] = 1

In [19]:
player_counts = pivot_data.sum(axis =0, skipna = True).sort_values(ascending=False)

In [20]:
pivot_data['Lady Gaga'].unique()

array([1., 0.])

In [21]:
df = pivot_data.reset_index(drop=True)

In [22]:
songs = pd.DataFrame(pivot_data.columns)

In [23]:
distance_matrix = cosine_similarity(df.T)
distance_matrix.shape

(17632, 17632)

In [41]:
distDF = pd.DataFrame(distance_matrix, columns=(df.columns), index=(df.columns))
distDF.shape[0]

17632

In [42]:
model = NearestNeighbors(n_neighbors=distDF.shape[0])
model.fit(distDF)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=17632, p=2,
         radius=1.0)

In [43]:
nearest_neighbours= model.kneighbors(distDF, return_distance=True)

In [45]:
nearest_neighbours[1].shape

(17632, 17632)

In [ ]:
NNDF = pd.DataFrame(distDF.columns[nearest_neighbours[1]], index=distDF.index)
top10=NNDF[NNDF.columns[1:11]]
top10

In [55]:
df.shape

(1892, 17632)

In [59]:
model_knn = NearestNeighbors(n_neighbors=df.shape[0], metric='cosine', algorithm='brute')
model_knn.fit(df)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=1892, p=2,
         radius=1.0)

In [60]:
nearest_neighbors= model_knn.kneighbors(df, return_distance=True)

In [86]:
nn= pd.DataFrame(nearest_neighbors[1])
nn.head()

,0,1,2,3,4,5,6,7,8,9,...,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891
0,0,881,228,1368,40,1203,1314,1117,112,247,...,957,962,965,966,970,971,973,977,985,1891
1,1,417,57,398,1171,1780,343,238,484,1362,...,637,636,635,634,633,632,631,630,642,1891
2,2,1361,416,1062,174,247,1433,59,537,716,...,954,957,958,966,978,985,990,993,1890,1891
3,3,1655,765,1609,1184,842,1824,371,1629,1362,...,932,942,1890,947,949,951,952,958,922,1891
4,4,851,776,1173,1010,1500,466,1615,1138,718,...,753,752,751,750,749,748,747,746,773,1891


In [ ]:
query_index = np.random.choice(df.shape[0])
distance, indices = model_knn.kneighbors(df.iloc[query_index, :].reshape(-1), n_neighbors = 6)

In [89]:
NNDF = pd.DataFrame(df.columns[nn], index=df.index)
top10=NNDF[df.columns[1:11]]
top10

AssertionError: Number of Block dimensions (3) must equal number of axes (2)

In [82]:
len(df.columns)

17632

In [83]:
len(nn)

1892